In [3]:
#libraries and tool importation
import numpy as np # linear algebra
import matplotlib.pylab as plt

import pandas as pd # data processing
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM, Bidirectional
from tensorflow.keras.layers import Dropout
import tensorflow as tf 
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from keras.layers import BatchNormalization, Dropout
from keras.layers import Dense, Activation
from tensorflow import keras

from sklearn.neighbors import KNeighborsClassifier  
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import GaussianNB

plt.rcParams["figure.figsize"] = (16,10) #Make the plots bigger by default
plt.rcParams["lines.linewidth"] = 2 #Setting the default line width
plt.style.use("ggplot")

In [6]:
#original emg data
df = pd.read_csv("/Users/dhruvaggarwal/Development/EEG-Classification/combine.csv")

In [7]:
# Assuming df is your pandas DataFrame
# You can replace "df" with the actual variable name of your DataFrame

# Remove rows where both "open" and "close" columns are zero
df_filtered = df[(df['open'] != 0) | (df['close'] != 0)]

# Reset the index after filtering
df_filtered = df_filtered.reset_index(drop=True)


In [33]:

# Combine the four class columns into a single column 'class_label'
class_columns = ['open', 'close', 'thumb', 'others']
df['class_label'] = df[class_columns].idxmax(axis=1)
df['class_label'] = df['class_label'].apply(lambda x: class_columns.index(x))

In [34]:
df

,index,time stamp,s1,s2,open,close,thumb,others,class_label
0,0,2023-08-19 16:47:03.066,-2.0,-2.0,1,0,0,0,0
1,1,2023-08-19 16:47:03.068,0.0,NaN,1,0,0,0,0
2,2,2023-08-19 16:47:03.068,0.0,0.0,1,0,0,0,0
3,3,2023-08-19 16:47:03.068,0.0,-1.0,1,0,0,0,0
4,4,2023-08-19 16:47:03.068,-1.0,1.0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...
23545,2600,2023-08-22 16:53:38.546,2.0,0.0,0,0,0,1,3
23546,2601,2023-08-22 16:53:38.748,-3.0,-6.0,0,0,0,1,3
23547,2602,2023-08-22 16:53:38.943,-1.0,4.0,0,0,0,1,3
23548,2603,2023-08-22 16:53:39.148,3.0,4.0,0,0,0,1,3


In [5]:
df.dtypes

index           int64
time stamp     object
s1            float64
s2            float64
open            int64
close           int64
thumb           int64
others          int64
dtype: object

In [36]:
df #print the first 5 rows of df

,index,time stamp,s1,s2,open,close,thumb,others,class_label
0,0,2023-08-19 16:47:03.066,-2.0,-2.0,1,0,0,0,0
1,1,2023-08-19 16:47:03.068,0.0,NaN,1,0,0,0,0
2,2,2023-08-19 16:47:03.068,0.0,0.0,1,0,0,0,0
3,3,2023-08-19 16:47:03.068,0.0,-1.0,1,0,0,0,0
4,4,2023-08-19 16:47:03.068,-1.0,1.0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...
23545,2600,2023-08-22 16:53:38.546,2.0,0.0,0,0,0,1,3
23546,2601,2023-08-22 16:53:38.748,-3.0,-6.0,0,0,0,1,3
23547,2602,2023-08-22 16:53:38.943,-1.0,4.0,0,0,0,1,3
23548,2603,2023-08-22 16:53:39.148,3.0,4.0,0,0,0,1,3


In [37]:
x = df.loc[:,["s1", "s2"]]
# y = df.loc[:,4:7]

In [38]:
x

,s1,s2
0,-2.0,-2.0
1,0.0,NaN
2,0.0,0.0
3,0.0,-1.0
4,-1.0,1.0
...,...,...
23545,2.0,0.0
23546,-3.0,-6.0
23547,-1.0,4.0
23548,3.0,4.0


In [39]:
y = df.loc[:,["class_label"]]

In [40]:
y

,class_label
0,0
1,0
2,0
3,0
4,0
...,...
23545,3
23546,3
23547,3
23548,3


In [26]:
#check of the dimensions
print(x.shape) 
print(y.shape) 
print(df.shape[0])

(17999, 2)
(17999, 2)
23549


In [27]:
y.dtypes

open     int64
close    int64
dtype: object

In [53]:
x.shape[1]

2

In [88]:
x = np.array(x)
y = np.array(y)

In [92]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42)



In [106]:
print(x_train.shape)
print(y_train.shape)

(17662, 2)
(17662, 4)


In [57]:

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense, Reshape


input_shape = (x.shape[1],)  # Shape of each sample in x_train

# Create the Sequential model
model = Sequential()

model.add(Reshape((input_shape[0], 1), input_shape=input_shape))  # Reshape input data
model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.2))  # Dropout rate = 20%

model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(units=50))
model.add(Dropout(0.2))

model.add(Dense(units=64))
model.add(Dense(units=128))

model.add(Dense(units=2, activation="softmax"))  # 4 as the output classes
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

# Print the model summary
model.summary()



Model: "sequential_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape_6 (Reshape)         (None, 2, 1)              0         
                                                                 
 lstm_31 (LSTM)              (None, 2, 50)             10400     
                                                                 
 dropout_28 (Dropout)        (None, 2, 50)             0         
                                                                 
 lstm_32 (LSTM)              (None, 2, 50)             20200     
                                                                 
 dropout_29 (Dropout)        (None, 2, 50)             0         
                                                                 
 lstm_33 (LSTM)              (None, 2, 50)             20200     
                                                                 
 dropout_30 (Dropout)        (None, 2, 50)           

In [11]:

# Convert 'open' and 'close' labels to binary format
df_filtered['open_close_binary'] = np.where(df_filtered['open'] == 0, 1, 0)

In [15]:
df_filtered

,index,time stamp,s1,s2,open,close,thumb,others,open_close_binary
0,0,2023-08-19 16:47:03.066,-2.0,-2.0,1,0,0,0,0
1,1,2023-08-19 16:47:03.068,0.0,NaN,1,0,0,0,0
2,2,2023-08-19 16:47:03.068,0.0,0.0,1,0,0,0,0
3,3,2023-08-19 16:47:03.068,0.0,-1.0,1,0,0,0,0
4,4,2023-08-19 16:47:03.068,-1.0,1.0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...
17995,1995,2023-08-22 16:51:37.013,-6.0,1.0,0,1,0,0,1
17996,1996,2023-08-22 16:51:37.213,9.0,0.0,0,1,0,0,1
17997,1997,2023-08-22 16:51:37.415,-8.0,-3.0,0,1,0,0,1
17998,1998,2023-08-22 16:51:37.614,5.0,3.0,0,1,0,0,1


In [42]:
import pandas as pd

# Assuming x is your input data
x = x.fillna(0)


In [27]:
x

,s1,s2
0,-2.0,-2.0
1,0.0,0.0
2,0.0,0.0
3,0.0,-1.0
4,-1.0,1.0
...,...,...
17995,-6.0,1.0
17996,9.0,0.0
17997,-8.0,-3.0
17998,5.0,3.0


In [44]:
y

,class_label
0,0
1,0
2,0
3,0
4,0
...,...
23545,3
23546,3
23547,3
23548,3


In [43]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import RandomForestClassifier

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Create and train the Random Forest model
model = RandomForestClassifier(n_estimators=100,random_state=42)
model.fit(X_train_scaled, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test_scaled)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

print(f'Accuracy: {accuracy}')
print(f'Classification Report:\n{classification_rep}')

/var/folders/m9/nn6zgszd65v8mmskfjh44b_r0000gn/T/ipykernel_53674/679602326.py:17: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X_train_scaled, y_train)


Accuracy: 0.6647558386411889
Classification Report:
              precision    recall  f1-score   support

           0       0.71      0.92      0.80      1822
           1       0.65      0.80      0.72      1782
           2       0.10      0.01      0.01       356
           3       0.22      0.04      0.06       750

    accuracy                           0.66      4710
   macro avg       0.42      0.44      0.40      4710
weighted avg       0.56      0.66      0.59      4710

